In [1]:
import pandas as pd
import numpy as np
# import torch
# from sklearn.metrics.pairwise import cosine_similarity
# from tqdm import tqdm
# from spacy.lang.fa import Persian
# from spacy.lang.he import Hebrew
# from spacy.lang.el import Greek
import os
# from POS.POSTagger import POSTagger

# def ActivateModel(modelname="bert-base-multilingual-cased"):
#     from transformers import AutoConfig, AutoTokenizer, AutoModel

#     # Activate model and tokenizer

#     # "HooshvareLab/bert-fa-base-uncased"

#     config = AutoConfig.from_pretrained(modelname)
#     tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
#     model = AutoModel.from_pretrained(modelname)
#     return config, tokenizer, model

def LoadBible(parquet_filename):
    # Read pre-processed farsi bible as pandas dataframe
    return pd.read_parquet(f"transformations/{parquet_filename}.parquet")

def GenerateBibleWords(bible_df, target):     
    for bk in sorted(set(bible_df.book)):
        for ic in bible_df.loc[bible_df.book == bk].idx_chapter:
            for iv in bible_df.loc[(bible_df.book == bk) & (bible_df.idx_chapter == ic)].idx_verse:
                # Prepare subset of verse(s) for feeding to model
                bible_words = (
                    bible_df
                    .loc[
                        (bible_df.book == bk) &
                        (bible_df.idx_chapter == ic) &
                        (bible_df.idx_verse == iv)
                    ]
                )
                if target == "idx_word":
                    yield bible_words.idx_word.to_list(), (bk, ic, iv)
                else:
                    bible_words = (
                        bible_words
                        .groupby(["book", "idx_chapter", "idx_verse", "idx_word"])[target]
                        .agg(" ".join)
                        .to_list()
                    )
                    split_bible_words = [a for sub in [s.split("/") for s in bible_words] for a in sub]
                    yield split_bible_words, (bk, ic, iv)

# def GetWordEmbeddings(words, model, tokenizer):

#     def mean_pooling(model_output, attention_mask):
#         token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#         input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#         sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
#         sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#         return sum_embeddings / sum_mask
        
#     # Tokenize sentences
#     encoded_input = tokenizer(words, padding=True, truncation=False, return_tensors='pt')

#     # Compute token embeddings
#     with torch.no_grad():
#         model_output = model(**encoded_input)
#     embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

#     # words_embeddings_df = pd.DataFrame({"word": words, "embeddings": embeddings.tolist()})
#     return [np.array(e).reshape(1,-1) for e in embeddings.tolist()]

# # How to only process non stopwords and punctuation?
# def max_cos_sim(row, model, tokenizer):
#     ser = row.copy()

#     ser["embeddings_nmv"] = GetWordEmbeddings(ser.word, model, tokenizer)
#     ser["embeddings_original"] = GetWordEmbeddings(ser.eng_word, model, tokenizer)
#     # if len(row["word"]) > 1 or row["word"] in ['آ', 'و', '\u200c', '\u200f']:
#     similarity = pd.DataFrame(
#         [
#             (
#                 ixo, 
#                 ixn,
#                 wn,
#                 ptn,
#                 cosine_similarity(eo, en)
#             ) 
#             for ixo, eo 
#             in zip(
#                 ser.idx_word_original, 
#                 ser.embeddings_original
#             )
#             for ixn, wn, ptn, en 
#             in zip(
#                 ser.idx_word_nmv,
#                 ser.word,
#                 ser.pos_tag,
#                 ser.embeddings_nmv
#             )
#         ]
#     )
#     similarity.columns = ["idx_word_original", "idx_wordmatch_nmv", "wordmatch_nmv", "pos_tag_nmv", "cosine_similarity"]
#     similarity["helper"] = 0
#     out = (
#         similarity
#         .sort_values("cosine_similarity")
#         .drop_duplicates(
#             subset="idx_word_original",
#             keep="last"
#         )
#         .groupby("helper")
#         [["idx_wordmatch_nmv", "wordmatch_nmv", "pos_tag_nmv", "cosine_similarity"]]
#         .agg(list)
#     )
#     # print(out.squeeze())
#     return out.squeeze()

# def GenerateMatches(
#     idx_gen,
#     nmv_gen,
#     orig_gen,
#     strongs_gen
# ):
# # figure out how to iterate through in parallel and avoid restarting generator each time. 
#     for idx_verse, nmv_verse, orig_verse, strongs in zip(
#         idx_gen,
#         nmv_gen,
#         orig_gen,
#         strongs_gen
#     ):
#         nmv_embeddings = GetWordEmbeddings(nmv_verse[0], model, tokenizer)
#         nmv_embeddings["idx_word"] = idx_verse[0]
#         orig_embeddings = GetWordEmbeddings(orig_verse[0], model, tokenizer)
#         orig_embeddings["strongs"] = strongs[0]
#         orig_embeddings[["idx_word", "similarity"]] = orig_embeddings.apply(max_cos_sim, axis=1, args=[nmv_embeddings])
#         nmv_embeddings = (
#             nmv_embeddings
#             .set_index("idx_word")
#             .join(
#                 orig_embeddings
#                 .drop(columns="embeddings")
#                 .set_index("idx_word"),
#                 how="left",
#                 rsuffix="_orig"
#             )
#         )
#         nmv_embeddings = (
#             nmv_embeddings
#             .reset_index()
#             .sort_values(by="similarity")
#             .drop_duplicates(subset="idx_word", keep="last")
#             .sort_values(by="idx_word")
#         )
#         nmv_embeddings[["book", "idx_chapter", "idx_verse"]] = idx_verse[1]
#         nmv_embeddings = (
#             nmv_embeddings
#             .reset_index()
#             .set_index(["book", "idx_chapter", "idx_verse"])
#         )
#         yield nmv_embeddings.drop(columns="embeddings")


In [28]:
from spacy.lang.fa import Persian
from spacy.lang.en import English

fa = Persian()
eng = English()

In [47]:
def GetWords(x):
    return [a for sub in (s.split("/") for s in list(x)) for a in sub]

def ListWOFaPunctuation(x):
    return[a for a in list(x) if not fa.vocab[a].is_punct and not fa.vocab[a].is_stop]

nt = ["Matthew", "Mark", "Luke", "John", "Acts", "Romans", "I Corinthians", "II Corinthians", "Galatians", "Ephesians", "Philippians", "Colossians", "I Thessalonians", "II Thessalonians", "I Timothy", "II Timothy", "Titus", "Philemon", "Hebrews", "James", "I Peter", "II Peter", "I John", "II John", "III John", "Jude", "Revelation of John"]

original = LoadBible("original")
original = (
    original
    .groupby(["book", "idx_chapter", "idx_verse"])
    [["eng_word", "strongs", "morphology"]]
    .agg(list)
)

nmv = LoadBible("nmv_pos_tagged")

nmv = (
    nmv
    .groupby(["book", "idx_chapter", "idx_verse"])
    [["word"]]
    .agg(list)
)


print(original.loc[("Genesis",0,0)])
print(nmv.loc[("Genesis",0,0)])


eng_word         [ב/ראשית, ברא, אלהים, את, ה/שמים, ו/את, ה/ארץ]
strongs       [Hb/H7225, H1254, H430, H853, Hd/H8064, Hc/H85...
morphology    [HR/Ncfsa, HVqp3ms, HNcmpa, HTo, HTd/Ncmpa, HC...
Name: (Genesis, 0, 0), dtype: object
word    [در, آغاز, ،, خدا, آسمانها, و, زمین, را, آفرید...
Name: (Genesis, 0, 0), dtype: object


In [48]:
def ListWOEnPunctuation(x):
    return[a for a in list(x) if not eng.vocab["None" if a==None else a].is_punct]

kjv = LoadBible("KJV")


In [49]:
kjv = (
    kjv
    .groupby(["book", "idx_chapter", "idx_verse"])
    [["eng_word", "strongs", "morphology"]]
    .agg(list)
)
print(kjv.loc[("Genesis",0,0)])


eng_word      [In the beginning, God, created, the heaven, a...
strongs       [H7225, H430, H853 H1254, H8064, H853, H776, N...
morphology         [None, None, TH8804, None, None, None, None]
Name: (Genesis, 0, 0), dtype: object


In [26]:
from simalign import SentenceAligner

aligner = SentenceAligner()


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-11-03 10:37:17,443 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [63]:

trg = kjv.loc[("Acts",0,0),"eng_word"]
src = nmv.loc[("Acts",0,0),"word"]

alignments = aligner.get_word_aligns(src, trg)

for method in alignments:
    method_alignments = {srci:kjv.loc[("Acts",0,0),"strongs"][trgi] for srci,trgi in alignments[method]}
    print(method, ":", [list(filter(lambda item: item is not None, [word, method_alignments[src.index(word)]])) if src.index(word) in method_alignments  else [word] for word in src])

mwmf : [['من', 'G4160'], ['کتاب', 'G3056'], ['نخست', 'G4413'], ['خود', 'G3056'], ['را', 'G4160'], ['،'], ['ای', 'G5599'], ['تِئوفیلوس', 'G2321'], ['،'], ['در', 'G4413'], ['باب', 'G4012'], ['همۀ'], ['اموری', 'G3956'], ['تألیف', 'G4160'], ['کردم'], ['که', 'G3739'], ['عیسی', 'G3588 G2424'], ['به', 'G4160'], ['عمل', 'G4160'], ['نمودن'], ['و', 'G2532'], ['تعلیم', 'G1321'], ['دادنشان', 'G5037'], ['آغاز', 'G756'], ['کرد']]
inter : [['من', 'G4160'], ['کتاب', 'G3056'], ['نخست', 'G4413'], ['خود'], ['را'], ['،'], ['ای', 'G5599'], ['تِئوفیلوس', 'G2321'], ['،'], ['در'], ['باب', 'G4012'], ['همۀ'], ['اموری', 'G3956'], ['تألیف'], ['کردم'], ['که', 'G3739'], ['عیسی', 'G3588 G2424'], ['به', 'G4160'], ['عمل'], ['نمودن'], ['و', 'G2532'], ['تعلیم', 'G1321'], ['دادنشان'], ['آغاز', 'G756'], ['کرد']]
itermax : [['من', 'G4160'], ['کتاب', 'G3056'], ['نخست', 'G4413'], ['خود', 'G3056'], ['را'], ['،'], ['ای', 'G5599'], ['تِئوفیلوس', 'G2321'], ['،'], ['در', 'G4413'], ['باب', 'G4012'], ['همۀ'], ['اموری', 'G3956'], ['

In [68]:
from tqdm import tqdm
def Align(row):
    trg = row.eng_word
    src = row.word
    alignments = aligner.get_word_aligns(src, trg)
    method_alignments = {srci:row.strongs[trgi] for srci,trgi in alignments["itermax"]}
    return [list(filter(lambda item: item is not None, [word, method_alignments[src.index(word)]])) if src.index(word) in method_alignments  else [word] for word in src]

combined_df = kjv.join(nmv)
combined_df.head()

tqdm.pandas(desc="Aligning words for each verse in the Bible")
combined_df["fa_strongs"] = combined_df.progress_apply(Align, axis=1)

Aligning words for each verse in the Bible:   1%|          | 320/31102 [06:26<9:56:52,  1.16s/it] 

In [17]:
print(trg)

word    [در, آغاز, ،, خدا, آسمانها, و, زمین, را, آفرید...
Name: (Genesis, 0, 0), dtype: object


In [ ]:
nt = ["Matthew", "Mark", "Luke", "John", "Acts", "Romans", "I Corinthians", "II Corinthians", "Galatians", "Ephesians", "Philippians", "Colossians", "I Thessalonians", "II Thessalonians", "I Timothy", "II Timothy", "Titus", "Philemon", "Hebrews", "James", "I Peter", "II Peter", "I John", "II John", "III John", "Jude", "Revelation of John"]

original = LoadBible("original")
original_ot = (
    original
    .loc[~original.book.isin(nt)]
    .set_index(["book", "idx_chapter", "idx_verse", "idx_word"])
    [["eng_word", "strongs", "morphology"]]
)

nmv = LoadBible("nmv_pos_tagged")

nmv_ot = (
    nmv
    .loc[~nmv.book.isin(nt)]
    .set_index(["book", "idx_chapter", "idx_verse", "idx_word"])
    [["word", "pos_tag"]]
)

fa_nlp = Persian()
he_nlp = Hebrew()
gr_nlp = Greek()

# fa_pos_tagger = POSTagger("model/perpos.model")

config, tokenizer, model = ActivateModel(modelname=os.path.join(os.environ["USERPROFILE"], "farsi-strongs", "outputs"))


OSError: Error no file named pytorch_model.bin, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory C:\Users\saaam\farsi-strongs\outputs.

In [4]:
# tqdm.pandas(desc="POS tagging farsi words")
# nmv["pos_tag"] = nmv.word.progress_apply(fa_pos_tagger.parse)
# nmv.head()

POS tagging farsi words: 100%|██████████| 31103/31103 [01:46<00:00, 291.51it/s]


idx_word  \
book idx_chapter idx_verse                                                      
Acts 0           0          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                 1          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                 2          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                 3          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                 4          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                                                         word  \
book idx_chapter idx_verse                                                      
Acts 0           0          [من, کتاب, نخست, خود, را, ،, ای, تِئوفیلوس, ،,...   
                 1          [تا, روزی, که, به, واسطۀ, روح‌القدس, دستورهایی...   
                 2          [او, پس, از, رنج, کشیدن, ،, خویشتن, را, بر, آن...   
                 3          [یک, بار, در, حین, صرف, غذا, بدیشان, امر, فرمو...   
                 4          [زیرا, یحیی, با, آب, تعمید, می‌داد, ،, امّا, چ...   

                                                                      pos_tag  
book idx_chapter idx_verse                                                     
Acts 0           0          [(من, RES), (کتاب, RES), (نخست, RES), (خود, RE...  
                 1          [(تا, CONJ), (روزی, N), (که, CONJ), (به, P), (...  
                 2          [(او, PRO), (پس, P), (از, P), (رنج, Ne), (کشید...  
                 3          [(یک, NUM), (بار, N), (در, P), (حین, Ne), (صرف...  
                 4          [(زیرا, CONJ), (یحیی, N), (با, P), (آب, N), (ت...

In [19]:
original["exclude"] = original.eng_word.apply(
    lambda x:any(
        [
            he_nlp.vocab[x].is_stop, 
            he_nlp.vocab[x].is_punct, 
            gr_nlp.vocab[x].is_stop, 
            gr_nlp.vocab[x].is_punct,
        ]
    )
)

nmv["exclude"] = nmv.word.apply(lambda x:any([fa_nlp.vocab[x].is_stop, fa_nlp.vocab[x].is_punct]))



In [31]:
original_subset = (
    original
    .loc[~original.exclude]
    .reset_index()
    .groupby(["book", "idx_chapter", "idx_verse"])
    [["eng_word", "strongs", "morphology", "idx_word"]]
    .agg(list)
)

nmv_subset = (
    nmv
    .loc[~nmv.exclude]
    .reset_index()
    .groupby(["book", "idx_chapter", "idx_verse"])
    [["word", "pos_tag", "idx_word"]]
    .agg(list)
)

In [49]:
merged_df = (
    original_subset
    .join(
        nmv_subset,
        how="left",
        lsuffix="_original",
        rsuffix="_nmv"
    )
).loc[("Genesis",0,0):("Genesis",0,1),:]


In [50]:
merged_df

eng_word  \
book    idx_chapter idx_verse                                                      
Genesis 0           0                 [ב/ראשית, ברא, אלהים, ה/שמים, ו/את, ה/ארץ]   
                    1          [ו/ה/ארץ, תהו, ו/בהו, ו/חשך, פני, תהום, ו/רוח,...   

                                                                         strongs  \
book    idx_chapter idx_verse                                                      
Genesis 0           0          [Hb/H7225, H1254, H430, Hd/H8064, Hc/H853, Hd/...   
                    1          [Hc/Hd/H776, H8414, Hc/H922, Hc/H2822, H6440, ...   

                                                                      morphology  \
book    idx_chapter idx_verse                                                      
Genesis 0           0          [HR/Ncfsa, HVqp3ms, HNcmpa, HTd/Ncmpa, HC/To, ...   
                    1          [HC/Td/Ncbsa, HNcmsa, HC/Ncmsa, HC/Ncmsa, HNcb...   

                                                  idx_word_original  \
book    idx_chapter idx_verse                                         
Genesis 0           0                            [0, 1, 2, 4, 5, 6]   
                    1          [0, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13]   

                                                                            word  \
book    idx_chapter idx_verse                                                      
Genesis 0           0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   

                                                                         pos_tag  \
book    idx_chapter idx_verse                                                      
Genesis 0           0          [[آغاز, N], [خدا, N], [آسمانها, N], [زمین, N],...   
                    1          [[زمین, Ne], [بی‌شکل, AJ], [خالی, AJ], [تاریکی...   

                                                       idx_word_nmv  
book    idx_chapter idx_verse                                        
Genesis 0           0                               [1, 3, 4, 6, 8]  
                    1          [0, 1, 3, 7, 10, 13, 14, 15, 16, 18]

In [51]:

tqdm.pandas(desc="matching words")
merged_df[["idx_wordmatch_nmv", "wordmatch_nmv", "pos_tag_nmv", "cosine_similarity"]] = (
    merged_df
    .progress_apply(
        max_cos_sim, 
        args=(model, tokenizer), 
        axis=1
    )
)

merged_df

matching words: 100%|██████████| 2/2 [00:00<00:00,  3.95it/s]

idx_wordmatch_nmv                                   [3, 3, 3, 3, 3, 3]
wordmatch_nmv                           [خدا, خدا, خدا, خدا, خدا, خدا]
pos_tag_nmv          [[خدا, N], [خدا, N], [خدا, N], [خدا, N], [خدا,...
cosine_similarity    [[[0.5750017952637303]], [[0.5802883873468905]...
Name: 0, dtype: object


matching words: 100%|██████████| 2/2 [00:01<00:00,  1.48it/s]

idx_wordmatch_nmv          [14, 14, 14, 14, 14, 14, 0, 14, 14, 14, 14]
wordmatch_nmv        [خدا, خدا, خدا, خدا, خدا, خدا, زمین, خدا, خدا,...
pos_tag_nmv          [[خدا, N], [خدا, N], [خدا, N], [خدا, N], [خدا,...
cosine_similarity    [[[0.5539344012703646]], [[0.569367715437644]]...
Name: 0, dtype: object


eng_word  \
book    idx_chapter idx_verse                                                      
Genesis 0           0                 [ב/ראשית, ברא, אלהים, ה/שמים, ו/את, ה/ארץ]   
                    1          [ו/ה/ארץ, תהו, ו/בהו, ו/חשך, פני, תהום, ו/רוח,...   

                                                                         strongs  \
book    idx_chapter idx_verse                                                      
Genesis 0           0          [Hb/H7225, H1254, H430, Hd/H8064, Hc/H853, Hd/...   
                    1          [Hc/Hd/H776, H8414, Hc/H922, Hc/H2822, H6440, ...   

                                                                      morphology  \
book    idx_chapter idx_verse                                                      
Genesis 0           0          [HR/Ncfsa, HVqp3ms, HNcmpa, HTd/Ncmpa, HC/To, ...   
                    1          [HC/Td/Ncbsa, HNcmsa, HC/Ncmsa, HC/Ncmsa, HNcb...   

                                                  idx_word_original  \
book    idx_chapter idx_verse                                         
Genesis 0           0                            [0, 1, 2, 4, 5, 6]   
                    1          [0, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13]   

                                                                            word  \
book    idx_chapter idx_verse                                                      
Genesis 0           0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   

                                                                         pos_tag  \
book    idx_chapter idx_verse                                                      
Genesis 0           0          [[آغاز, N], [خدا, N], [آسمانها, N], [زمین, N],...   
                    1          [[زمین, Ne], [بی‌شکل, AJ], [خالی, AJ], [تاریکی...   

                                                       idx_word_nmv  \
book    idx_chapter idx_verse                                         
Genesis 0           0                               [1, 3, 4, 6, 8]   
                    1          [0, 1, 3, 7, 10, 13, 14, 15, 16, 18]   

                                                         idx_wordmatch_nmv  \
book    idx_chapter idx_verse                                                
Genesis 0           0                                   [3, 3, 3, 3, 3, 3]   
                    1          [14, 14, 14, 14, 14, 14, 0, 14, 14, 14, 14]   

                                                                   wordmatch_nmv  \
book    idx_chapter idx_verse                                                      
Genesis 0           0                             [خدا, خدا, خدا, خدا, خدا, خدا]   
                    1          [خدا, خدا, خدا, خدا, خدا, خدا, زمین, خدا, خدا,...   

                                                                     pos_tag_nmv  \
book    idx_chapter idx_verse                                                      
Genesis 0           0          [[خدا, N], [خدا, N], [خدا, N], [خدا, N], [خدا,...   
                    1          [[خدا, N], [خدا, N], [خدا, N], [خدا, N], [خدا,...   

                                                               cosine_similarity  
book    idx_chapter idx_verse                                                     
Genesis 0           0          [[[0.5750017952637303]], [[0.5802883873468905]...  
                    1          [[[0.5539344012703646]], [[0.569367715437644]]...

In [52]:
merged_df.explode(
    column=[
        "idx_word_original",
        "eng_word",
        "strongs",
        "morphology",
        "idx_wordmatch_nmv",
        "wordmatch_nmv",
        "pos_tag_nmv",
        "cosine_similarity"
    ]
)

eng_word     strongs   morphology  \
book    idx_chapter idx_verse                                     
Genesis 0           0          ב/ראשית    Hb/H7225     HR/Ncfsa   
                    0              ברא       H1254      HVqp3ms   
                    0            אלהים        H430       HNcmpa   
                    0           ה/שמים    Hd/H8064    HTd/Ncmpa   
                    0             ו/את     Hc/H853        HC/To   
                    0            ה/ארץ     Hd/H776    HTd/Ncbsa   
                    1          ו/ה/ארץ  Hc/Hd/H776  HC/Td/Ncbsa   
                    1              תהו       H8414       HNcmsa   
                    1            ו/בהו     Hc/H922     HC/Ncmsa   
                    1            ו/חשך    Hc/H2822     HC/Ncmsa   
                    1              פני       H6440       HNcbpc   
                    1             תהום       H8415       HNcbsa   
                    1            ו/רוח    Hc/H7307     HC/Ncbsc   
                    1            אלהים        H430       HNcmpa   
                    1            מרחפת       H7363      HVprfsa   
                    1              פני       H6440       HNcbpc   
                    1            ה/מים    Hd/H4325    HTd/Ncmpa   

                              idx_word_original  \
book    idx_chapter idx_verse                     
Genesis 0           0                         0   
                    0                         1   
                    0                         2   
                    0                         4   
                    0                         5   
                    0                         6   
                    1                         0   
                    1                         2   
                    1                         3   
                    1                         4   
                    1                         6   
                    1                         7   
                    1                         8   
                    1                         9   
                    1                        10   
                    1                        12   
                    1                        13   

                                                                            word  \
book    idx_chapter idx_verse                                                      
Genesis 0           0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    0                          [آغاز, خدا, آسمانها, زمین, آفرید]   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   
                    1          [زمین, بی‌شکل, خالی, تاریکی, ژرفا, روح, خدا, س...   

                                                                         pos_tag  \
book    idx_chapter idx_verse                                                      
Genesis 0           0

Worked really well! except it used אלהים twice - for خدا and آفرید. Find a way to limit to one use per word.
Has a problem with names and confusing Jesus with Christ. Check titles of God.

Low similarity matches for prepositions etc. Consider removing stopwords to avoid incorrect matches.

Resolve by Part of Speech (POS) tagging and looking for matches in morphological codes that match the POS tag.
Also option to use Named Entity Recognition (NER) to match proper nouns. Could transliterate instead of translating and match that way if names are still a problem.
Need to find a tokenizer that deals with farsi multi-word expressions e.g. verbs that are a couplet of noun and verb like Doha mikonam - prayer I do.
SpaCy library farsi language resources might help do this to prepare the dataframe for feeding into BERT in this notebook.

Resolve matching multiple times by finding best farsi match to original word instead of other way round. Removing stopwords from original before matching will also help the above issue.
Use idx_word to match the resulting match back to the farsi text in the right order and allow for added farsi words without a match in the original.

In [1]:
a = [1,2,3,4,5,6]
b = [7,8,9,10,11,12]

c = [str(ai)+"-"+str(bi) for ai in a for bi in b]

In [2]:
c

['1-7',
 '1-8',
 '1-9',
 '1-10',
 '1-11',
 '1-12',
 '2-7',
 '2-8',
 '2-9',
 '2-10',
 '2-11',
 '2-12',
 '3-7',
 '3-8',
 '3-9',
 '3-10',
 '3-11',
 '3-12',
 '4-7',
 '4-8',
 '4-9',
 '4-10',
 '4-11',
 '4-12',
 '5-7',
 '5-8',
 '5-9',
 '5-10',
 '5-11',
 '5-12',
 '6-7',
 '6-8',
 '6-9',
 '6-10',
 '6-11',
 '6-12']